In [1]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.vgg16 import preprocess_input

img_size = 224
Data_directory= 'D:\\Hope Our Handson\\11.Deep Learning\\Face Mask detection\\dataset\\'

def load_images_from_folder(folder_path, label):
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, (img_size, img_size))
            images.append(img)
            labels.append(label)
    return images, labels

with_mask_path = os.path.join(Data_directory, 'with_mask')
without_mask_path = os.path.join(Data_directory, 'without_mask')

x_with_mask, y_with_mask = load_images_from_folder(with_mask_path, 1)
x_without_mask, y_without_mask = load_images_from_folder(without_mask_path, 0)

x = np.array(x_with_mask + x_without_mask)
y = np.array(y_with_mask + y_without_mask)

# Preprocess the images
x = preprocess_input(x.astype(np.float32))  # for VGG16
y = y.astype(np.float32)

# Split into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)#stratify=y->Preserves class distribution across train and test splits


C:\Users\HP\anaconda3\envs\Aidl\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 2.2.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


ImportError: DLL load failed while importing _multiarray_umath: The specified module could not be found.

ImportError: numpy.core.multiarray failed to import

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

# Load VGG16 Predefined model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

# Freeze base layers and we can train model to learn for extra features
for layer in base_model.layers:
    layer.trainable = False


#For Sequential CNN We have used Sequential APi here we used Functional APi like x = Layer(...)(previous_layer)
x = Flatten()(base_model.output)  	#x = Layer(...)(previous_layer)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=Adam(learning_rate=1e-3),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train
history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.1)


In [ ]:
from sklearn.metrics import classification_report

# Predict
y_pred = (model.predict(x_test) > 0.5).astype("int32")

# Report
print(classification_report(y_test, y_pred, target_names=["Without Mask", "With Mask"]))

# Save model
model.save("vgg16_mask_detector_model.h5")


In [ ]:
print(x_test[2].shape)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Select the image from test set
img = x_test[5]
img_input = np.expand_dims(img, axis=0)  # Add batch dimension

# Make prediction
pred = model.predict(img_input)
label = "With Mask" if pred[0][0] > 0.5 else "Without Mask"

# Show the image with prediction label
plt.imshow(img)  # No reshape if already (224,224,3)
plt.title(f"Prediction: {label}")
plt.axis('off')
plt.show()